# Linear Solver Performance: XDG Stokes, MPI-parallel (Development at $ J=13824 $)



### Part 1, Benchmark Setup and Execution

In [1]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

Execution Date/time is 28/03/2023 10:13:17


In [2]:
//#r "c:/Users/kummer/Documents/BoSSS-kummer/public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
//#r "c:/Users/kummer/Documents/BoSSS-kummer/public/src/L4-application/BoSSSpad/bin/Debug/net6.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.Utils.Geom;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.GridGen;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [3]:
string PROJECT_NAME = "LinslvPerfPar-XdgSokesJ13824";//System.Environment.GetEnvironmentVariable("LinslvPerfPar") ?? "LinslvPerfPar"; // this allows to modify the project name for testing purposes
wmg.Init(PROJECT_NAME);
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

In [4]:
//wmg.ResetProject(deleteSessions:true);

In [5]:
ExecutionQueues

index,type,value
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,MiniBatchProcessor client @C:\Users\flori\AppData\Local\BoSSS-LocalJobs
1,BoSSS.Application.BoSSSpad.SlurmClient,"SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006"
2,BoSSS.Application.BoSSSpad.SlurmClient,"SlurmClient Lb2-specialPrj-Jenkins : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006"


In [6]:
GetDefaultQueue()

DeploymentBaseDirectory,C:\Users\flori\AppData\Local\BoSSS-LocalJobs
DeployRuntime,False
RuntimeLocation,win\amd64
Name,<null>
DotnetRuntime,dotnet
BatchInstructionDir,<null>
AllowedDatabasesPaths,"[ C:\Users\flori, C:\ ]"


In [7]:
 //Lb2-specialPrj 
 

## Utility definitions

In [8]:
static class Utils {
    // DOF per cell in 3D for one variable
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
    */
}

## Init grids and save to database

Note: The dimension of the domain $(-1,1)^3$ are assumed to be **centimeters**!
In this benchmark, realistic physical values are used, i.e. densities and viscosities of 
- water for Phase A (inside the droplet)
- air for Phase B (surounding)
The droplet is assumed to be in the millimeter range; therefore, all lenght-, area-, and volume-related properties 
must also be set in centimeters, e.g. the density of water is $10^{-3} \text{kg}/\text{cm}^3$.

In [9]:
wmg.Grids

Opening existing database '\\wsl.localhost\Ubuntu-22.04\home\florian\lichtb-scratch\bosss_databases\LinslvPerfPar-XdgSokesJ13824'.
Opening existing database '\\wsl.localhost\Ubuntu-22.04\home\florian\lichtb-scratch\jenkins\databases\LinslvPerfPar-XdgSokesJ13824'.


#0: { Guid = 3df53a86-ff4f-4209-aede-aa0faa32d665; Name = LinslvPerfPar-XdgSokesJ13824-XdgStokes_J262144; Cell Count = 262144; Dim = 3 }
#1: { Guid = 8fe00748-8f30-4bfd-b95d-30987af9da21; Name = LinslvPerfPar-XdgSokesJ13824-XdgStokes_J110592; Cell Count = 110592; Dim = 3 }
#2: { Guid = b462ed9f-4e54-419a-8beb-7ae72e92aa5d; Name = LinslvPerfPar-XdgSokesJ13824-XdgStokes_J32768; Cell Count = 32768; Dim = 3 }
#3: { Guid = 5efbd710-dd9b-48a6-a840-eca0254cd8ad; Name = LinslvPerfPar-XdgSokesJ13824-XdgStokes_J13824; Cell Count = 13824; Dim = 3 }
#4: { Guid = 5dbd78b7-f710-48ef-8d78-d31129978d45; Name = LinslvPerfPar-XdgSokesJ13824-XdgStokes_J4096; Cell Count = 4096; Dim = 3 }
#5: { Guid = 5555af48-fd28-40a1-9d25-0d01bb956269; Name = LinslvPerfPar-XdgSokesJ13824-XdgStokes_J512; Cell Count = 512; Dim = 3 }


In [10]:
int[] Resolutions_3D = new int[] { 24 };
string[] GridNameS = new string[Resolutions_3D.Length];
var ggcS = new (GridGenControl C, int MPIsize)[Resolutions_3D.Length];

for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] _xNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] _yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] _zNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (_xNodes.Length - 1)*(_yNodes.Length - 1)*(_zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "-XdgStokes_J" + J);
    GridNameS[cnt] = GridName;  
    
    if(wmg.Grids.Where(grd => grd.Name.Contains(GridName)).Count() <= 0) {
        int NoOfProcs = (int) Math.Min(182, Math.Max(1, Math.Ceiling(J/200000.0)));
        Console.WriteLine("Must create: " + GridName + " with " + NoOfProcs + " processors.");
        
        var C = new GridGenControl();
        ggcS[cnt] = (C, NoOfProcs);
        C.SetDatabase(wmg.DefaultDatabase);
        
        C.GridName = GridName;

        C.GridBlocks = new GridGenControl.MeshBlock[] {
            new GridGenControl.Cartesian3D() {
                xNodes = _xNodes,
                yNodes = _yNodes,
                zNodes = _zNodes
            }
        };

        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -1-1e-8, -2, -2 }, new double[] { -1+1e-8, +2, +2 }), 
            "wall_left"));
        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { +1-1e-8, -2, -2 }, new double[] { +1+1e-8, +2, +2 }), 
            "wall_right"));
        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -2, -1-1e-8, -2 }, new double[] { +2, -1+1e-8, +2 }), 
            "wall_front"));
        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -2, +1-1e-8, -2 }, new double[] { +2, +1+1e-8, +2 }), 
            "wall_back"));
        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -2, -2, -1-1e-8 }, new double[] { +2, +2, -1+1e-8 }), 
            "wall_top"));
        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -2, -2, +1-1e-8 }, new double[] { +2, +2, +1+1e-8 }), 
            "wall_bottom"));
        
        
        C.SessionName = "GridCreation-" + GridName;
    } else {
        Console.WriteLine("Found grid: " + GridName);
    }
}

Found grid: LinslvPerfPar-XdgSokesJ13824-XdgStokes_J13824


In [11]:
foreach(var tt in ggcS) {
    if(tt.C != null) {
        Console.WriteLine(" Submitting: " + tt.C.SessionName); 
        var j = tt.C.CreateJob();
        j.RetryCount = 2;
        j.NumberOfMPIProcs = tt.MPIsize;
        j.Activate();
    }
}

In [12]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish
wmg.AllJobs

All jobs finished.


In [13]:
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    int J = Res*Res*Res;
    
    Console.WriteLine("Searching for grid with " + J + " cells");
    grids[cnt] = wmg.Grids.FirstOrDefault(grd => grd.Name.Contains(GridNameS[cnt])); // grid must be present now
    
    if(grids[cnt] != null) {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");

        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    } else {
        Console.Error.WriteLine("missing: J = " + J);
    }
}

Searching for grid with 13824 cells
Found Grid: { Guid = 5efbd710-dd9b-48a6-a840-eca0254cd8ad; Name = LinslvPerfPar-XdgSokesJ13824-XdgStokes_J13824; Cell Count = 13824; Dim = 3 }


In [15]:
var GrdJ13824 = (grids[0] as GridProxy).RealGrid as BoSSS.Foundation.Grid.Classic.GridCommons;
GrdJ13824

{ Guid = 5efbd710-dd9b-48a6-a840-eca0254cd8ad; Name = LinslvPerfPar-XdgSokesJ13824-XdgStokes_J13824; Cell Count = 13824; Dim = 3 }

In [14]:
int For2(double[] X) {
    double x = X[0];
    return (x > 0) ? 1 : 0;
}
int For4(double[] X) {
    double y = X[1];
    return ((y > 0) ? 1 : 0) + For2(X)*2;
}
int For8(double[] X) {
    double z = X[2];
    return ((z > 0) ? 1 : 0) + For4(X)*2;
}

In [16]:
var _GrdJ13824 = GrdJ13824.CloneAs();

In [17]:
_GrdJ13824.AddPredefinedPartitioning("For2", For2);
_GrdJ13824.AddPredefinedPartitioning("For4", For4);
_GrdJ13824.AddPredefinedPartitioning("For8", For8);

In [18]:
_GrdJ13824.PredefinedGridPartitioning

key,value
For2,BoSSS.Foundation.Grid.Classic.GridCommons+GridPartitioningVector
For4,BoSSS.Foundation.Grid.Classic.GridCommons+GridPartitioningVector
For8,BoSSS.Foundation.Grid.Classic.GridCommons+GridPartitioningVector


In [23]:
var tst = "For8";
Console.WriteLine(_GrdJ13824.PredefinedGridPartitioning[tst].CellToRankMap.Min());
Console.WriteLine(_GrdJ13824.PredefinedGridPartitioning[tst].CellToRankMap.Max());
for(int i = 0; i < 8; i++)
    Console.WriteLine(_GrdJ13824.PredefinedGridPartitioning[tst].CellToRankMap.Where(rnk => rnk == i).Count());

0
7
1728
1728
1728
1728
1728
1728
1728
1728


In [24]:
databases[0].SaveGrid(ref _GrdJ13824, force:true);

In [32]:
grids[0] = _GrdJ13824;

## Setup Control Object for a Solver Run

In [34]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [35]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

### Setup of Parameter Study

Polynomial degrees to test:

In [36]:
int[] PolyDegS = new int[] { 2, 3, 5 };

In [38]:
int[] MPIsizes = new int[] { 2, 4, 8, 16, 32, 48 };

Loop over all combinations of parameters and define a control object for each combo:

In [31]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [39]:
var controls = new List<(XNSE_Control ctrl, int NoOfProcs)>();
LinearSolverCode solver_name = LinearSolverCode.exp_Kcycle_schwarz;
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {
foreach(int MPIsize in MPIsizes) {
    
    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*3 + Np_p;
    if(J / MPIsize < 16) {
        // less than 16 cells per processor - to low for a multigrid.
        continue; 
    }
    if(DOF/MPIsize > 500000) {
        // not interested in doing more then 500'000 DOFs per processor
        continue;
    } 
    
    
    // Control Instance, grid, DG degree, etc.
    // =======================================
    
    XNSE_Control C = new XNSE_Control();
    controls.Add((C, MPIsize));
       
    string caseName = string.Format("XdgStokes-J{0}_p{1}_Sz{2}", J, k, MPIsize);
    Console.WriteLine("setting up: " + caseName);
    C.SessionName        = caseName;
    
    C.SetGrid(grd);
    C.savetodb = true;
    C.SetDGdegree(k);
    
    // Phys. Parameters
    // ================
    
    // Species A: Water; Species B: Air
    C.PhysicalParameters.rho_A             = 1.0; //1e-3; //     kg / cm³
    C.PhysicalParameters.rho_B             = 1.0; //1.2e-6; //   kg / cm³
    C.PhysicalParameters.mu_A              = 1.0; //1e-5; //      kg / cm / sec
    C.PhysicalParameters.mu_B              = 1.0; //17.1e-8; //   kg / cm / sec
    C.PhysicalParameters.Sigma             = 72.75e-3; // kg / sec²   
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    
     //C.Option_LevelSetEvolution                          = LevelSetEvolution.FastMarching;
    C.AdvancedDiscretizationOptions.SST_isotropicMode   = SurfaceStressTensor_IsotropicMode.Curvature_Projected;
    //C.AdvancedDiscretizationOptions.ViscosityMode       = ViscosityMode.Standard;
    C.AdvancedDiscretizationOptions.FilterConfiguration = CurvatureAlgorithms.FilterConfiguration.NoFilter;
    double r     = 0.5;
    double nonsp = 0.5;

    C.AddInitialValue("Phi", new Formula($"X => (X[0]/{r*nonsp}).Pow2() + (X[1]/{r}).Pow2() + (X[2]/{r}).Pow2() - 1", false));
    
    C.LSContiProjectionMethod = BoSSS.Solution.LevelSetTools.ContinuityProjectionOption.None;
    //C.CutCellQuadratureType   = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.ComputeEnergyProperties = false;

    
    // Solver Stuff
    // ============
    
    C.LinearSolver           = solver_name.GetConfig();
    if(C.LinearSolver is IterativeSolverConfig isc) {
        isc.ConvergenceCriterion = 1e-8;
    }
    C.NoOfMultigridLevels = 100;
    C.TracingNamespaces = "BoSSS.Solution";
    C.LevelSet_ConvergenceCriterion     = 1e-6;

    C.MemoryInstrumentationLevel = ilPSP.Tracing.MemoryInstrumentationLevel.None;
    C.DynamicLoadBalancing_RedistributeAtStartup = true;
    C.DynamicLoadBalancing_On = true;

   
    
    // Timestepping / Instationary
    // ===========================

    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
    //C.dtFixed = 0.01;
}
}
}

setting up: XdgStokes-J13824_p2_Sz2
setting up: XdgStokes-J13824_p2_Sz4
setting up: XdgStokes-J13824_p2_Sz8
setting up: XdgStokes-J13824_p2_Sz16
setting up: XdgStokes-J13824_p2_Sz32
setting up: XdgStokes-J13824_p2_Sz48
setting up: XdgStokes-J13824_p3_Sz2
setting up: XdgStokes-J13824_p3_Sz4
setting up: XdgStokes-J13824_p3_Sz8
setting up: XdgStokes-J13824_p3_Sz16
setting up: XdgStokes-J13824_p3_Sz32
setting up: XdgStokes-J13824_p3_Sz48
setting up: XdgStokes-J13824_p5_Sz2
setting up: XdgStokes-J13824_p5_Sz4
setting up: XdgStokes-J13824_p5_Sz8
setting up: XdgStokes-J13824_p5_Sz16
setting up: XdgStokes-J13824_p5_Sz32
setting up: XdgStokes-J13824_p5_Sz48


In [43]:
foreach(var (ctrl, MpiSz) in controls) {
    XNSE_Control ctrl_clone;
    
    {
    ctrl_clone = (XNSE_Control) AppControl.Deserialize(ctrl.Serialize());

    ctrl_clone.DynamicLoadBalancing_CellCostEstimators.Clear();
    ctrl_clone.DynamicLoadBalancing_RedistributeAtStartup = false;
    ctrl_clone.DynamicLoadBalancing_On = false;
    ctrl_clone.GridPartType = GridPartType.METIS;

    ctrl_clone.SaveToFile(System.IO.Path.Combine(@"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net6.0\control",
                    "Control-" + ctrl.SessionName + "-static_metis.obj" 
                     ));

    }

     {
    ctrl_clone = (XNSE_Control) AppControl.Deserialize(ctrl.Serialize());

    ctrl_clone.DynamicLoadBalancing_CellCostEstimators.Clear();
    ctrl_clone.DynamicLoadBalancing_CellCostEstimators.Add(new BoSSS.Application.XNSE_Solver.Loadbalancing.XNSECellCostEstimator());
    ctrl_clone.DynamicLoadBalancing_RedistributeAtStartup = true;
    ctrl_clone.DynamicLoadBalancing_On = true;
    ctrl_clone.GridPartType = GridPartType.METIS;

    ctrl_clone.SaveToFile(System.IO.Path.Combine(@"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net6.0\control",
                    "Control-" + ctrl.SessionName + "-dyn_metis.obj" 
                     ));

    }

    if((new int[] {2, 4, 8}).Contains(MpiSz)) {
    ctrl_clone = (XNSE_Control) AppControl.Deserialize(ctrl.Serialize());

    ctrl_clone.DynamicLoadBalancing_CellCostEstimators.Clear();
    ctrl_clone.DynamicLoadBalancing_RedistributeAtStartup = false;
    ctrl_clone.DynamicLoadBalancing_On = false;
    ctrl_clone.GridPartType = GridPartType.Predefined;
    ctrl_clone.GridPartOptions = "For" + MpiSz;

    ctrl_clone.SaveToFile(System.IO.Path.Combine(@"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net6.0\control",
                    "Control-" + ctrl.SessionName + "-predef.obj" 
                     ));

    }
}

Total number of simulations:

In [ ]:
controls.Count

In [ ]:
// Assert that the location of the fluid phases is as desired:
NUnit.Framework.Assert.Negative(controls[0].ctrl.InitialValues["Phi"].Evaluate(new double[]{0,0.0,0}, 0.0), "phase A (negative) must be inside");
NUnit.Framework.Assert.Positive(controls[0].ctrl.InitialValues["Phi"].Evaluate(new double[]{0,4.0,0}, 0.0), "phase B (positive) must be inside");

In [ ]:
// Basic checks on the multigrid configuration
foreach(var (ctrl, sz) in controls) {
    NUnit.Framework.Assert.Greater(ctrl.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
    if(ctrl.LinearSolver is OrthoMGSchwarzConfig osc)
        NUnit.Framework.Assert.Greater(osc.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
}

In [ ]:
//string path = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\Xstokes";
//foreach(var ctrl in controls) {
//    ctrl.savetodb = false;
//    ctrl.SaveToFile(System.IO.Path.Combine(path, ctrl.SessionName + ".obj"));
//}

## Launch Jobs

Use the default queue defined on this machine:

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = GetDefaultQueue();
myBatch

In [ ]:
//wmg.ResetProject(deleteJobs:true, deleteSessions:true);

In [ ]:
foreach((var ctrl, int MPIsize) in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var j = ctrl.CreateJob();
    j.RetryCount = 1;
    j.NumberOfMPIProcs = MPIsize;
    j.Activate(myBatch);
    //ctrl.RunBatch();
}

### Wait for Completion and Check Job Status

In [ ]:
//wmg.AllJobs["XdgStokes-J13824_p2_Sz2"].ShowOutput()

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
wmg.Sessions.Where(sess => sess.Name.StartsWith("XdgStokes-J"))

In [ ]:
int succJob = wmg.AllJobs.Values.Where(job => !job.Name.Contains("GridCreation") && job.Status == JobStatus.FinishedSuccessful).Count();
int succSess = wmg.Sessions.Where(Si => Si.Name.Contains("XdgStokes-J") && (Si.SuccessfulTermination == true)).Count();
(succJob, succSess)

In [ ]:
int failJob = wmg.AllJobs.Values.Where(job => !job.Name.Contains("GridCreation") && job.Status != JobStatus.FinishedSuccessful).Count();
int failSess = wmg.Sessions.Where(Si => Si.Name.Contains("XdgStokes-J") && (Si.SuccessfulTermination == false)).Count();
(failJob, failSess)

In [ ]:
NUnit.Framework.Assert.AreEqual(succJob + failJob, controls.Count);
//NUnit.Framework.Assert.AreEqual(succJob, succSess);
//NUnit.Framework.Assert.AreEqual(failJob, failSess);

In [ ]:
var NoSuccess = wmg.AllJobs.Values
                        .Where(job => job.Status != JobStatus.FinishedSuccessful)
                        .ToArray();
NoSuccess

In [ ]:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + ((fail.LatestDeployment?.DeploymentDirectory?.FullName) ?? " no deployment directory"));

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("XdgStokes") && !Si.Name.Contains("GridCreation") &&
                                        (Si.SuccessfulTermination == false
                                        || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0)).ToArray();
FailedSessions

In [ ]:
Job GetJob4Session(ISessionInfo s) {
    return wmg.AllJobs.Values.FirstOrDefault(job => wmg.SessionInfoJobCorrelation(s, job));
}

In [ ]:
GetJob4Session(FailedSessions[0])

In [ ]:
GetJob4Session(FailedSessions[1])

In [ ]:
GetJob4Session(FailedSessions[0]).AllSessions

In [ ]:
using(var wrt = new System.IO.StreamWriter("FailReport.txt")) {
    foreach(var s in FailedSessions) {
        wrt.WriteLine(s.ToString());
        wrt.WriteLine(s.GetStdout());
        wrt.WriteLine(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>");
        string errString = s.GetStderr();
        string[] lines = errString.Split('\n');
        string truncatedString = string.Join("\n", lines.Take(10));
        wrt.WriteLine(truncatedString);
        if(lines.Length > 10)
           wrt.WriteLine("  ... truncated ...");
        wrt.WriteLine("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<");
        
        for(int i = 0; i < 10; i++)
            wrt.WriteLine("********************************************************");
    }

}

In [ ]:
wmg.AllJobs["XdgStokes-J13824_p5_Sz48"].AllSessions

In [ ]:
FailedSessions[0]

In [ ]:
/*foreach(var j in wmg.AllJobs.Values) {
    if(j.AllSessions.Length > 1) {
        Console.WriteLine(j);
        foreach(var s in j.AllSessions) {
            Console.WriteLine("   " + s);
        }
    }
    
    if(j.AllSessions.Length < 0) {
        Console.WriteLine("NO SESSION: " + j);
    }
}*/

In [ ]:
//wmg.Sessions

In [ ]:
(ISessionInfo si, Plot2Ddata mem)[] mpiMem = wmg.Sessions.Select(s => (s,s.GetMPItotalMemory())).ToArray();

In [ ]:
wmg.Sessions[0].GetSessionDirectory()

In [ ]:
//var di = new System.IO.DirectoryInfo(@"\\dc1\userspace\kummer\cluster\LinslvPerfPar-XdgSokesJ13824\sessions\7a343be7-b51c-44e3-898a-805550cffb15");
//var memI = new SessionMemtrace(di);

In [83]:
//mpiMem[5].Item2.dataGroups[0].Name = "Tot Mem [MegB] at 32 cores (2)";

In [84]:
mpiMem.Select(tt => (tt.Item1.Name, tt.Item2.dataGroups[0].Name))

index,Item1,Item2
0,XdgStokes-J13824_p2_Sz2,Tot Mem [MegB] at 2 cores
1,XdgStokes-J13824_p5_Sz48,Tot Mem [MegB] at 48 cores
2,XdgStokes-J13824_p5_Sz48,Tot Mem [MegB] at 48 cores (2)
3,XdgStokes-J13824_p5_Sz32,Tot Mem [MegB] at 32 cores
4,XdgStokes-J13824_p5_Sz48,Tot Mem [MegB] at 48 cores (3)
5,XdgStokes-J13824_p5_Sz32,Tot Mem [MegB] at 32 cores (2)
6,XdgStokes-J13824_p3_Sz2,Tot Mem [MegB] at 2 cores
7,XdgStokes-J13824_p2_Sz4,Tot Mem [MegB] at 4 cores
8,XdgStokes-J13824_p3_Sz16,Tot Mem [MegB] at 16 cores
9,XdgStokes-J13824_p5_Sz16,Tot Mem [MegB] at 16 cores


In [96]:
var multiplot = new Plot2Ddata[3,1];
foreach(var tt in new [] { (0, "p2"), (1, "p3"), (2, "p5") }) {
    multiplot[tt.Item1, 0] = new Plot2Ddata();
    var pXplot = mpiMem.Where(qq => qq.Item1.Name.Contains(tt.Item2));
    multiplot[tt.Item1, 0] = multiplot[tt.Item1, 0].Merge(pXplot.Select(qq => qq.Item2).ToArray());
    
    int iPlt = 0;
    foreach(var xy in multiplot[tt.Item1, 0].dataGroups) {
        xy.Format.Style = Styles.Lines;
        xy.Format.SetLineColorFromIndex(iPlt);
        iPlt++;
        
    }
}
//multiplot.PlotNow();
multiplot.PlotCairolatex(ySize:20.0).WriteMinimalCompileableExample("mem.tex");

Using gnuplot: C:\Users\kummer\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


In [130]:
int ParseNoOfCores(string name) {
   var regex = new System.Text.RegularExpressions.Regex(@"at (\d+)");
   var match = regex.Match(name);
   if (match.Success) 
       return int.Parse(match.Groups[1].Value);
   else
       throw new Exception();
}


(double,double)[] memVsCores(int iRow) {
    return multiplot[iRow,0].dataGroups
      .OrderBy(dataGroup => ParseNoOfCores(dataGroup.Name))
      .Select(dataGroup => ((double)ParseNoOfCores(dataGroup.Name),dataGroup.Values.Max()))
      .ToArray();
}

In [126]:
memVsCores

index,Item1,Item2
0,2,83872.37890625
1,4,88552.8984375
2,8,106274.81640625
3,16,121208.62109375
4,32,115165.76953125
5,48,120589.61328125


In [135]:
Plot(memVsCores(0).Select(tt => tt.Item1), memVsCores(0).Select(tt => tt.Item2), "P=2", "-xb",
     memVsCores(1).Select(tt => tt.Item1), memVsCores(1).Select(tt => tt.Item2), "P=3", "-or",
     memVsCores(2).Select(tt => tt.Item1), memVsCores(2).Select(tt => tt.Item2), "P=5", "-*m")

Using gnuplot: C:\Users\kummer\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 200000 
 
 
 
 
 400000 
 
 
 
 
 600000 
 
 
 
 
 800000 
 
 
 
 
 1x10 6 
 
 
 
 
 1.2x10 6 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3 
 
 
 
 
 
 
 
 
 
 
 
 
 P=5 
 
 
 P=5